In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
!pip install kaggle

In [ ]:
import json

kaggle_credentials = json.load(open("kaggle.json"))

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = kaggle_credentials['username']
os.environ['KAGGLE_KEY'] = kaggle_credentials['key']

In [ ]:
!kaggle datasets download -d arjuntejaswi/plant-village

Dataset URL: https://www.kaggle.com/datasets/arjuntejaswi/plant-village
License(s): unknown
 99% 325M/329M [00:02<00:00, 105MB/s]
100% 329M/329M [00:02<00:00, 119MB/s]


In [ ]:
!ls

kaggle.json  plant-village.zip	sample_data


In [ ]:
from zipfile import ZipFile

with ZipFile("plant-village.zip", 'r') as zip_ref:
    zip_ref.extractall()


In [ ]:
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1):
        super().__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size, padding=padding, groups=in_channels)
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x

class FasterCNN(nn.Module):
    def __init__(self, num_classes):
        super(FasterCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            DepthwiseSeparableConv(3, 64),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            DepthwiseSeparableConv(64, 128),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2, 2),
            DepthwiseSeparableConv(128, 256),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2, 2),
        )

        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc_layers = nn.Sequential(
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.global_avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x


In [ ]:
import torch
torch.cuda.empty_cache()


In [ ]:
# ✅ Step 1: Define Transformations
train_transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Reduce image size
    transforms.RandomResizedCrop(128, scale=(0.7, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

test_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


In [ ]:
# ✅ Step 2: Load Dataset & Split into Train and Test
full_dataset = datasets.ImageFolder(root="PlantVillage", transform=train_transform)

train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_data, test_data = random_split(full_dataset, [train_size, test_size])

In [ ]:
# Assign correct transformations
train_data.dataset.transform = train_transform
test_data.dataset.transform = test_transform


ttrain_loader = DataLoader(train_data, batch_size=64, shuffle=True, num_workers=4)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False, num_workers=4)


num_classes = len(full_dataset.classes)  # Get number of classes
print(f"Dataset split: {train_size} train, {test_size} test.")
print(f"Number of Classes: {num_classes}")

Dataset split: 16510 train, 4128 test.
Number of Classes: 15


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
# ✅ Step 3: Define CNN Model (More Layers for Better Accuracy)
import torch.nn.functional as F
def mixup_data(x, y, alpha=1.0):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    index = torch.randperm(x.size(0)).to(x.device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

In [ ]:
# ✅ Step 4: Initialize Model, Loss & Optimizer
model = FasterCNN(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)

In [ ]:
# ✅ Step 5: Enable Mixed Precision (FP16 Training)
scaler = torch.cuda.amp.GradScaler()

<ipython-input-18-bd0b9110d002>:2: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


In [ ]:
# ✅ Step 6: Train Model
num_epochs = 30  # ✅ Reduced epochs (faster training)
best_accuracy = 0.0
early_stop_count = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        with torch.cuda.amp.autocast():  # ✅ Mixed Precision
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()

    scheduler.step()

    # ✅ Validate Model
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {accuracy:.2f}%")

    # ✅ Save Best Model
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), "best_model.pth")
        print("✔ Model Saved!")
        early_stop_count = 0
    else:
        early_stop_count += 1

    if early_stop_count > 5:  # ✅ Early stopping
        print("🔴 Early Stopping!")
        break

# ✅ Step 7: Final Model Evaluation
print(f"✅ Best Accuracy: {best_accuracy:.2f}%")

<ipython-input-19-8ca2526ee4b4>:14: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():  # ✅ Mixed Precision


Epoch [1/30], Loss: 1.0310, Accuracy: 73.57%
✔ Model Saved!
Epoch [2/30], Loss: 0.6171, Accuracy: 77.42%
✔ Model Saved!
Epoch [3/30], Loss: 0.4678, Accuracy: 77.64%
✔ Model Saved!
Epoch [4/30], Loss: 0.3706, Accuracy: 66.11%
Epoch [5/30], Loss: 0.3128, Accuracy: 80.50%
✔ Model Saved!
Epoch [6/30], Loss: 0.2621, Accuracy: 83.33%
✔ Model Saved!
Epoch [7/30], Loss: 0.2363, Accuracy: 76.94%
Epoch [8/30], Loss: 0.2069, Accuracy: 79.09%
Epoch [9/30], Loss: 0.1788, Accuracy: 76.79%
Epoch [10/30], Loss: 0.1674, Accuracy: 81.40%
Epoch [11/30], Loss: 0.1467, Accuracy: 86.70%
✔ Model Saved!
Epoch [12/30], Loss: 0.1293, Accuracy: 86.99%
✔ Model Saved!
Epoch [13/30], Loss: 0.1253, Accuracy: 88.95%
✔ Model Saved!
Epoch [14/30], Loss: 0.1154, Accuracy: 90.87%
✔ Model Saved!
Epoch [15/30], Loss: 0.1107, Accuracy: 89.66%
Epoch [16/30], Loss: 0.1039, Accuracy: 87.60%
Epoch [17/30], Loss: 0.0926, Accuracy: 87.89%
Epoch [18/30], Loss: 0.0904, Accuracy: 85.83%
Epoch [19/30], Loss: 0.0827, Accuracy: 88.61%


In [ ]:
from google.colab import files
files.download('best_model.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>